# Function
All the function here is for support our main program

In [ ]:
%matplotlib inline 

import import_ipynb
import Twitter
import sys 
import pandas as pd
from IPython.display import display
import requests
import ipywidgets as widgets
import ipython_blocking
import matplotlib
import webbrowser

trend = Twitter.trend()

def find_availiable_trend(usrPlace):
    for i in trend:
        if i['name'] == usrPlace:
            return i['woeid']
        else:
            continue
            
    return f'{usrPlace} is not available'

def get_trendlist(trendid):
    trendlist = Twitter.trend_place(trendid)
    trend_df = pd.DataFrame(data = trendlist[0]['trends'])
    trend_df = trend_df.sort_values(by = 'tweet_volume',ascending=False)
    trend_df = trend_df.filter(items = ['name','tweet_volume'])
    return trend_df.head(10)

def tweet2df(tweets):
    result_df = pd.DataFrame()
    result_df['screen_name'] = [tweet.user.screen_name for tweet in tweets]
    result_df['tweet'] = [tweet.text for tweet in tweets]
    result_df['time'] = [tweet.created_at for tweet in tweets]
    result_df['location'] = [tweet.user.location for tweet in tweets]
    result_df['retweet_count'] = [tweet.retweet_count for tweet in tweets]
    result_df['tweet_id'] = [tweet.id for tweet in tweets]
    result_df = result_df.sort_values(by = 'retweet_count',ascending=False)
    
    return result_df

def GetSentiment(text):
    # todo write code to return dict of sentiment for text
    url = 'http://text-processing.com/api/sentiment/'
    options = { 'text' : text }
    response = requests.post(url, data = options)
    sentiment = response.json()
    return sentiment

def pie_graph(sentiment):
    labels = 'pos', 'neg', 'neutral'
    colors = ['r', 'g','b']
    plt.pie(sentiment, labels=labels, colors=colors,startangle=90, autopct='%.1f%%')
    plt.axis('equal')
    return plt.show()

def viewOnTwitter(result_df):
    print('enter the ID to view it on Twitter.com')
    text = widgets.Text()

    twitter_buttion = widgets.Button(
        value=False,
        description='Go to Twitter.com',
    )
    output = widgets.Output()

    def ToTwitter(b):
        with output:
            twitter_id = result_df.at[int(text.value), 'tweet_id']
            webbrowser.open(f'https://twitter.com/Interior/status/{twitter_id}')

    go2Twitter = widgets.VBox(children=[text, twitter_buttion,output])
    twitter_buttion.on_click(ToTwitter)
    return go2Twitter